In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from surprise.model_selection import train_test_split,cross_validate
from surprise import SVD, accuracy
from surprise import KNNBasic
import os
from tensorflow.keras import backend as K
import tables

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from surprise import SVDpp, Reader, Dataset
from sklearn.preprocessing import LabelEncoder

2023-04-30 20:34:34.367413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 20:34:35.685501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### TASTE PROFILE DATA

In [2]:
taste_profile = pd.read_csv('/home/016651544/datasets/train_triplets.txt',sep='\t',header=None,names = ['user_id','song_id','play_count'],nrows = 2000000)#
taste_profile.shape

(2000000, 3)

In [3]:
taste_profile.head()

,user_id,song_id,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [4]:
# Compute quartiles for the play_count of users and year of publication columns
q1_play_count = taste_profile['play_count'].quantile(0.25)
q3_play_count = taste_profile['play_count'].quantile(0.75)
iqr_play_count = q3_play_count - q1_play_count
lower_play_count = q1_play_count - 1.5 * iqr_play_count
upper_play_count = q3_play_count + 1.5 * iqr_play_count
print(q1_play_count, q3_play_count,lower_play_count, upper_play_count)

# Identify and remove outliers based on IQR
taste_profile = taste_profile[(taste_profile['play_count'] >= lower_play_count) & (taste_profile['play_count'] <= upper_play_count)]
taste_profile.shape

1.0 3.0 -2.0 6.0


(1834519, 3)

In [5]:
taste_profile.song_id.nunique()

188992

In [6]:
df = taste_profile.copy()

### METADATA

In [7]:
# Convert the table to a Pandas DataFrame
metadata_table_df = pd.read_csv('/home/016651544/datasets/track_metadata.csv')
metadata_table_df.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,NaN,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0


In [8]:
song_ids = taste_profile["song_id"].unique().astype(str)
metadata_df = pd.merge(pd.DataFrame(song_ids,columns=['song_id']),metadata_table_df,on="song_id",how='left')
metadata_df.head()

,song_id,track_id,title,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,SOAKIMP12A8C130995,TRIQAUQ128F42435AD,The Cove,Thicker Than Water,ARC8CQZ1187B98DECA,ff6e677f-91dd-4986-a174-8db0474b1799,Jack Johnson,112.63955,0.832012,0.677482,0,2093263,-1,0
1,SOAPDEY12A81C210A9,TRIRLYL128F42539D1,Nothing from Nothing,To Die For,ARHYUI71187FB48366,8a7cf497-dc5c-4523-932d-3fcbc9a69d38,Billy Preston,153.59955,0.580555,0.482492,1974,3072847,-1,0
2,SOBBMDR12A8C13253B,TRMHBXZ128F4238406,Entre Dos Aguas,Flamenco Para Niños,ARC1SF21187FB51D0F,7bc8dd01-35ea-4b18-a64d-3a364dc76a52,Paco De Lucia,358.24281,0.633119,0.417718,1976,2631392,-1,0
3,SOBFNSP12AF72A0E22,TRYQMNI128F147C1C7,Under Cold Blue Stars,Under Cold Blue Stars,ARMS97V1187B99DD98,023d64c9-93db-4a20-8c5c-2efa1a53481a,Josh Rouse,266.34404,0.675339,0.509058,2002,413957,-1,0
4,SOBFOVM12A58A7D494,TRAHZNE128F9341B86,Riot Radio (Soundtrack Version),Nick & Norah's Infinite Playlist - Original Mo...,ARI4V0E1187B9B6FC2,87ebbe67-3910-4521-a418-4fe53eb912b7,The Dead 60s,141.42649,0.650281,0.451749,0,5674853,-1,0


### PREPROCESSING ON METADATA

In [9]:
metadata_df.replace("",np.nan,inplace=True)

In [10]:
metadata_df.isna().sum()

song_id                  0
track_id                 0
title                    2
release                  0
artist_id                0
artist_mbid           4279
artist_name              0
duration                 0
artist_familiarity       0
artist_hotttnesss        0
year                     0
track_7digitalid         0
shs_perf                 0
shs_work                 0
dtype: int64

In [11]:
values = {'song_hotttnesss': 0,'artist_familiarity': 0, 'title': "Unknown"}
metadata_df.fillna(value= values,inplace=True)
metadata_df.isna().sum()

song_id                  0
track_id                 0
title                    0
release                  0
artist_id                0
artist_mbid           4279
artist_name              0
duration                 0
artist_familiarity       0
artist_hotttnesss        0
year                     0
track_7digitalid         0
shs_perf                 0
shs_work                 0
dtype: int64

In [12]:
metadata_df.columns,metadata_df.dtypes

(Index(['song_id', 'track_id', 'title', 'release', 'artist_id', 'artist_mbid',
        'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
        'year', 'track_7digitalid', 'shs_perf', 'shs_work'],
       dtype='object'),
 song_id                object
 track_id               object
 title                  object
 release                object
 artist_id              object
 artist_mbid            object
 artist_name            object
 duration              float64
 artist_familiarity    float64
 artist_hotttnesss     float64
 year                    int64
 track_7digitalid        int64
 shs_perf                int64
 shs_work                int64
 dtype: object)

In [13]:
metadata_df.columns

Index(['song_id', 'track_id', 'title', 'release', 'artist_id', 'artist_mbid',
       'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
       'year', 'track_7digitalid', 'shs_perf', 'shs_work'],
      dtype='object')

### KNN, SVD

In [40]:
def get_recommendations_svd(scale ,metadata,model_name, train_data, user_id, n=10):
    """
    Get top n recommendations for the given user id using the specified model.
    """
    # Load the dataset and create a reader
    reader = Reader(rating_scale=scale)
    data = Dataset.load_from_df(train_data[['user_id', 'song_id', 'play_count']], reader)

    # Train the model
    if model_name == 'SVD':
        model = SVD(n_factors=5, n_epochs=5, lr_all=0.01, reg_all=0.02)
    elif model_name == 'KNN':
        model = KNNBasic(k=40, sim_options={'name': 'cosine', 'user_based': False})
    else:
        raise ValueError('Invalid model name')

    # Split the data into training and test sets
    trainset, testset = train_test_split(data, test_size=0.1, random_state=42)
#     trainset = data.build_full_trainset()
    model.fit(trainset)
    # Test the model on the test set and compute the accuracy metrics
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions)
    mae = accuracy.mae(predictions)
    
    print(" RMSE for {} ".format(model_name), rmse)
    print(" MAE for {} ".format(model_name), mae)
    # Get the user's item interactions
    user_items = train_data[train_data['user_id'] == user_id]['song_id'].unique()

    # Get the predictions for the user and sort them in descending order
    predictions = []
    for item_id in train_data['song_id'].unique():
        if item_id not in user_items:
            predictions.append((item_id, model.predict(user_id, item_id)[3]))
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    # Get the top n recommendations
    top_n = [x[0] for x in predictions[:n]]
    
    song_info = metadata_df[metadata_df['song_id'].isin(top_n)][[ 'title', 'artist_name', 'duration']]
    
    return song_info

In [41]:
# Get top 10 recommendations for user with ID 123 using the SVD model
scale = (1,6)
top_recommendations = get_recommendations_svd(scale,metadata_df,'SVD', df, 'b80344d063b5ccb3212f76538f3d9e43d87dca9e', n=10)
top_recommendations

RMSE: 1.2055
MAE:  0.8747
 RMSE for SVD  1.2055474903731247
 MAE for SVD  0.874707880729976


,title,artist_name,duration
1640,Decades Of Despair,Carnal Forge,182.33424
7893,Como Un Sueño (Am I Dreaming),Kat DeLuna,239.43791
10599,Bedroom Suite,Joe Christmas,185.78240
10836,Brother,Keisha White,251.42812
14776,Victoria (LP Version),Old 97's,231.28771
26952,Chapel Of Ghouls (Live),Morbid Angel,226.66404
31523,Tennessee (Pirate Radio Mix),ARRESTED DEVELOPMENT,371.59138
35084,TULENLIEKKI,M.A. Numminen,166.55628
38087,Prologue,Danny Elfman,176.84853
39439,Canción Para Luchar,León Gieco,283.61098


### Normalized Play counts

In [16]:
grp_df = df[['user_id','play_count']].groupby('user_id').sum().reset_index()
grp_df.columns = ['user_id','total_play_counts']
data_df=pd.merge(df,grp_df, on='user_id',how= 'left')
data_df['play_count_norm'] = data_df['play_count']/data_df['total_play_counts'] 

In [17]:
data_df.head()

,user_id,song_id,play_count,total_play_counts,play_count_norm
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,134,0.007463
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,134,0.007463
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,134,0.014925
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,134,0.007463
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,134,0.007463


In [18]:
scale = (0,1)
top_recommendations = get_recommendations_svd(scale,metadata_df,'SVD', data_df[['user_id','song_id','play_count_norm']].rename(columns = {'play_count_norm':'play_count'}), 'b80344d063b5ccb3212f76538f3d9e43d87dca9e', n=10)

RMSE: 0.0292
MAE:  0.0180
 RMSE for SVD  0.029217333935571303
 MAE for SVD  0.017986632253815017


In [19]:
top_recommendations

,title,artist_name,duration
54522,Black History Month (Josh Homme remix),Death From Above 1979,279.17016
54805,Me Gustan,DJ Bitman,175.85587
54858,Weapons of Mass Distortion,The Crystal Method,291.16036
84945,Honky Tonk Women,Billy Joel,197.61587
98339,Kiwi,Mustard Pimp,296.48934
102930,Wait For You,Sugababes,234.00444
116141,Suicide,The Devin Townsend Band,405.00200
133701,Here I am,Sexy Sadie,169.69098
171788,Sword And Shield (LP Version),Sister Hazel,305.84118
174415,Intro,Styles P,85.02812


In [20]:
final_data_df = taste_profile.copy()

In [21]:
final_data_df['unencoded_sid'] = final_data_df['song_id']
final_data_df['unencoded_uid'] = final_data_df['user_id']

In [22]:
from sklearn.preprocessing import LabelEncoder 
data_encoder_song = LabelEncoder()
data_encoder_user = LabelEncoder()

final_data_df['song_id'] = data_encoder_song.fit_transform(final_data_df['song_id'])
final_data_df['user_id'] = data_encoder_user.fit_transform(final_data_df['user_id'])

In [23]:
song_dict = dict(zip(final_data_df['unencoded_sid'], final_data_df['song_id']))
user_dict = dict(zip(final_data_df['unencoded_uid'], final_data_df['user_id']))
final_data_df.drop(columns =['unencoded_sid'], inplace=True)
final_data_df.drop(columns =['unencoded_uid'], inplace=True)
final_data_df.head(8)

,user_id,song_id,play_count
0,29973,3139,1
1,29973,4583,1
2,29973,8249,2
3,29973,9441,1
4,29973,9450,1
5,29973,12008,1
6,29973,13544,2
7,29973,14267,1


In [24]:
# Generate random indices
num_samples = len(final_data_df)
indices = np.random.permutation(num_samples)

# Split indices into train and test
train_size = int(0.8 * num_samples)
train_indices = indices[:train_size]
test_indices = indices[train_size:]

# Use indices to index data
train_data = final_data_df.iloc[train_indices]
test_data = final_data_df.iloc[test_indices]

In [25]:
# train_data.song_id= pd.Categorical(train_data.song_id) 
train_song_ids = train_data.song_id
# train_data.user_id= pd.Categorical(train_data.user_id) 
train_user_ids = train_data.user_id
train_output = train_data['play_count']


# test_data.song_id= pd.Categorical(test_data.song_id) 
test_song_ids = test_data.song_id
# test_data.user_id= pd.Categorical(test_data.user_id) 
test_user_ids = test_data.user_id
test_output = test_data['play_count']


### 4 Training a Neural Network

#### 4.1 Input dimensions for neural network

In [26]:
#nunique() will return the total number of unique items

num_songs = len(train_song_ids.unique())+len(test_song_ids.unique()) 
num_users =  len(train_user_ids.unique())+len(train_user_ids.unique())
song_input_dim = 1
user_input_dim = 1

num_songs,num_users,song_input_dim,user_input_dim

(267181, 83434, 1, 1)

In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import keras
from keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten
from keras import regularizers
from keras import Sequential

In [28]:
#Embedding layer for songs
input_songs = Input(shape=(song_input_dim,), name='songs_input')
embedding_songs = Embedding(input_dim=num_songs+1, output_dim=10, name='embedding_songs')(input_songs)
flatten_songs = Flatten(name='flatten_songs')(embedding_songs)

#Embedding layer for users
input_users = Input(shape=(user_input_dim,), name='users_input')
embedding_users = Embedding(input_dim=num_users+1, output_dim=10, name='embedding_users')(input_users)
flatten_users = Flatten(name='flatten_users')(embedding_users)


#Concatenation and Dense layers
concat_layer = Concatenate(name='concatenate_layer')([flatten_songs, flatten_users])
hidden_layer_1 = Dense(16, activation='relu', name='hidden_layer_1')(concat_layer)
hidden_layer_2 = Dense(8, activation='relu', name='hidden_layer_2')(hidden_layer_1)
output_layer = Dense(1, name='output_layer')(hidden_layer_2)

#Creating model
model1 = Model(inputs=[input_songs, input_users], outputs=output_layer)

#Model compilation
optimizer = Adam(lr=0.001, epsilon=1e-6, amsgrad=True)
model1.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model1.summary()

2023-04-30 20:35:28.746097: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 songs_input (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 users_input (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 embedding_songs (Embedding)    (None, 1, 10)        2671820     ['songs_input[0][0]']            
                                                                                                  
 embedding_users (Embedding)    (None, 1, 10)        834350      ['users_input[0][0]']            
                                                                                              

In [29]:
#Training model

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2)

history = model1.fit(
  [train_song_ids, train_user_ids], train_output, #
  batch_size=256, 
  epochs=50, 
  verbose=1,
  callbacks = [early_stopping],
  validation_split=0.2
)

Epoch 1/50
4587/4587 [==============================] - 61s 13ms/step - loss: 1.5822 - root_mean_squared_error: 1.2578 - val_loss: 1.4590 - val_root_mean_squared_error: 1.2079
Epoch 2/50
4587/4587 [==============================] - 58s 13ms/step - loss: 1.3591 - root_mean_squared_error: 1.1658 - val_loss: 1.4744 - val_root_mean_squared_error: 1.2142
Epoch 3/50
4587/4587 [==============================] - 83s 18ms/step - loss: 1.2614 - root_mean_squared_error: 1.1231 - val_loss: 1.5215 - val_root_mean_squared_error: 1.2335


In [30]:
from sklearn.metrics import mean_squared_error,r2_score
# Predict on test set
y_pred = model1.predict([test_song_ids,test_user_ids])

# Evaluate model
score = model1.evaluate([test_song_ids,test_user_ids], test_output, verbose=0)
rmse = np.sqrt(mean_squared_error(test_output, y_pred))
r2 = r2_score(test_output, y_pred)

print('Test RMSE:', rmse)
print('Test MSE:', score[0])
print('Test MAE:', score[1])


11466/11466 [==============================] - 13s 1ms/step
Test RMSE: 1.2350012782205022
Test MSE: 1.5252329111099243
Test MAE: 1.2350032329559326


### METADATA

In [31]:
# Convert the table to a Pandas DataFrame
metadata_table_df = pd.read_csv('../datasets/track_metadata.csv')
metadata_table_df.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,NaN,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0


In [32]:
song_ids = taste_profile["song_id"].unique().astype(str)
metadata_df = pd.merge(pd.DataFrame(song_ids,columns=['song_id']),metadata_table_df,on="song_id",how='left')
metadata_df.head()

,song_id,track_id,title,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,SOAKIMP12A8C130995,TRIQAUQ128F42435AD,The Cove,Thicker Than Water,ARC8CQZ1187B98DECA,ff6e677f-91dd-4986-a174-8db0474b1799,Jack Johnson,112.63955,0.832012,0.677482,0,2093263,-1,0
1,SOAPDEY12A81C210A9,TRIRLYL128F42539D1,Nothing from Nothing,To Die For,ARHYUI71187FB48366,8a7cf497-dc5c-4523-932d-3fcbc9a69d38,Billy Preston,153.59955,0.580555,0.482492,1974,3072847,-1,0
2,SOBBMDR12A8C13253B,TRMHBXZ128F4238406,Entre Dos Aguas,Flamenco Para Niños,ARC1SF21187FB51D0F,7bc8dd01-35ea-4b18-a64d-3a364dc76a52,Paco De Lucia,358.24281,0.633119,0.417718,1976,2631392,-1,0
3,SOBFNSP12AF72A0E22,TRYQMNI128F147C1C7,Under Cold Blue Stars,Under Cold Blue Stars,ARMS97V1187B99DD98,023d64c9-93db-4a20-8c5c-2efa1a53481a,Josh Rouse,266.34404,0.675339,0.509058,2002,413957,-1,0
4,SOBFOVM12A58A7D494,TRAHZNE128F9341B86,Riot Radio (Soundtrack Version),Nick & Norah's Infinite Playlist - Original Mo...,ARI4V0E1187B9B6FC2,87ebbe67-3910-4521-a418-4fe53eb912b7,The Dead 60s,141.42649,0.650281,0.451749,0,5674853,-1,0


### PREPROCESSING ON METADATA

In [33]:
metadata_df.replace("",np.nan,inplace=True)

In [34]:
metadata_df.isna().sum()

song_id                  0
track_id                 0
title                    2
release                  0
artist_id                0
artist_mbid           4279
artist_name              0
duration                 0
artist_familiarity       0
artist_hotttnesss        0
year                     0
track_7digitalid         0
shs_perf                 0
shs_work                 0
dtype: int64

In [35]:
values = {'song_hotttnesss': 0,'artist_familiarity': 0, 'title': "Unknown"}
metadata_df.fillna(value= values,inplace=True)
metadata_df.isna().sum()

song_id                  0
track_id                 0
title                    0
release                  0
artist_id                0
artist_mbid           4279
artist_name              0
duration                 0
artist_familiarity       0
artist_hotttnesss        0
year                     0
track_7digitalid         0
shs_perf                 0
shs_work                 0
dtype: int64

In [36]:
metadata_df.columns,metadata_df.dtypes

(Index(['song_id', 'track_id', 'title', 'release', 'artist_id', 'artist_mbid',
        'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
        'year', 'track_7digitalid', 'shs_perf', 'shs_work'],
       dtype='object'),
 song_id                object
 track_id               object
 title                  object
 release                object
 artist_id              object
 artist_mbid            object
 artist_name            object
 duration              float64
 artist_familiarity    float64
 artist_hotttnesss     float64
 year                    int64
 track_7digitalid        int64
 shs_perf                int64
 shs_work                int64
 dtype: object)

In [37]:
metadata_df.columns

Index(['song_id', 'track_id', 'title', 'release', 'artist_id', 'artist_mbid',
       'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
       'year', 'track_7digitalid', 'shs_perf', 'shs_work'],
      dtype='object')

In [38]:
def get_recommendations(user_id,model,taste_profile,metadata_df,data_encoder_user, data_encoder_song):
    song_id_df = pd.DataFrame(taste_profile.song_id.unique(),columns = ['song_id'])
    user_song_df = song_id_df.copy()
    user_song_df['user_id'] = user_id
    encoded_user = data_encoder_user.transform(user_song_df.user_id)
    encoded_song = data_encoder_song.transform(user_song_df.song_id)
    user_song_df['user_id'] = encoded_user
    user_song_df['song_id']= encoded_song
    listened_songs = list(final_data_df[final_data_df.user_id == user_id].song_id.unique())
    songs_to_predict = np.setdiff1d(song_id_df, listened_songs)
    data = pd.merge(user_song_df, final_data_df.drop(columns = 'user_id').drop_duplicates(), on =['song_id'], how = 'left')

    data['Predictions'] = model.predict([data.song_id, data.user_id])
    data['user_id'] = data_encoder_user.inverse_transform(data['user_id'])
    data['song_id'] =data_encoder_song.inverse_transform(data['song_id'])
    top_songs_rec = data[['user_id','song_id','Predictions']].sort_values(by = 'Predictions',ascending = False).song_id[:20].values
    song_info = metadata_df[metadata_df['song_id'].isin(top_songs_rec)][[ 'title', 'artist_name', 'duration']]
    return song_info.reset_index(drop=True)

In [39]:
user_id = 'b80344d063b5ccb3212f76538f3d9e43d87dca9e'
get_recommendations(user_id,model1,taste_profile,metadata_df,data_encoder_user, data_encoder_song)

12139/12139 [==============================] - 15s 1ms/step


,title,artist_name,duration
0,Decades Of Despair,Carnal Forge,182.33424
1,Brother,Keisha White,251.42812
2,Rise Above (Live) (Album Version),Misfits,164.46649
3,Tennessee (Pirate Radio Mix),ARRESTED DEVELOPMENT,371.59138


### Training on Likeness

In [47]:
final_data_df = taste_profile.copy()

In [48]:
final_data_df['unencoded_sid'] = final_data_df['song_id']
final_data_df['unencoded_uid'] = final_data_df['user_id']

In [49]:
grp_df = final_data_df[['user_id','play_count']].groupby('user_id').sum().reset_index()
grp_df.columns = ['user_id','total_play_counts']
final_data_df=pd.merge(final_data_df,grp_df, on='user_id',how= 'left')
final_data_df['play_count_norm'] = final_data_df['play_count']/final_data_df['total_play_counts'] 
final_data_df.head()

,user_id,song_id,play_count,unencoded_sid,unencoded_uid,total_play_counts,play_count_norm
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,SOAKIMP12A8C130995,b80344d063b5ccb3212f76538f3d9e43d87dca9e,134,0.007463
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,SOAPDEY12A81C210A9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,134,0.007463
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,SOBBMDR12A8C13253B,b80344d063b5ccb3212f76538f3d9e43d87dca9e,134,0.014925
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,SOBFNSP12AF72A0E22,b80344d063b5ccb3212f76538f3d9e43d87dca9e,134,0.007463
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,SOBFOVM12A58A7D494,b80344d063b5ccb3212f76538f3d9e43d87dca9e,134,0.007463


In [50]:
from sklearn.preprocessing import LabelEncoder 
data_encoder_song = LabelEncoder()
data_encoder_user = LabelEncoder()

final_data_df['song_id'] = data_encoder_song.fit_transform(final_data_df['song_id'])
final_data_df['user_id'] = data_encoder_user.fit_transform(final_data_df['user_id'])

In [51]:
song_dict = dict(zip(final_data_df['unencoded_sid'], final_data_df['song_id']))
user_dict = dict(zip(final_data_df['unencoded_uid'], final_data_df['user_id']))
final_data_df.drop(columns =['unencoded_sid'], inplace=True)
final_data_df.drop(columns =['unencoded_uid'], inplace=True)
final_data_df.head(8)

,user_id,song_id,play_count,total_play_counts,play_count_norm
0,29973,3139,1,134,0.007463
1,29973,4583,1,134,0.007463
2,29973,8249,2,134,0.014925
3,29973,9441,1,134,0.007463
4,29973,9450,1,134,0.007463
5,29973,12008,1,134,0.007463
6,29973,13544,2,134,0.014925
7,29973,14267,1,134,0.007463


In [52]:
# Generate random indices
num_samples = len(final_data_df)
indices = np.random.permutation(num_samples)

# Split indices into train and test
train_size = int(0.8 * num_samples)
train_indices = indices[:train_size]
test_indices = indices[train_size:]

# Use indices to index data
train_data = final_data_df.iloc[train_indices]
test_data = final_data_df.iloc[test_indices]

In [53]:
# train_data.song_id= pd.Categorical(train_data.song_id) 
train_song_ids = train_data.song_id
# train_data.user_id= pd.Categorical(train_data.user_id) 
train_user_ids = train_data.user_id
train_output = train_data['play_count_norm']


# test_data.song_id= pd.Categorical(test_data.song_id) 
test_song_ids = test_data.song_id
# test_data.user_id= pd.Categorical(test_data.user_id) 
test_user_ids = test_data.user_id
test_output = test_data['play_count_norm']


### 4 Training a Neural Network

#### 4.1 Input dimensions for neural network

In [54]:
#nunique() will return the total number of unique items

num_songs = len(train_song_ids.unique())+len(test_song_ids.unique()) 
num_users =  len(train_user_ids.unique())+len(train_user_ids.unique())
song_input_dim = 1
user_input_dim = 1

num_songs,num_users,song_input_dim,user_input_dim

(267169, 83440, 1, 1)

In [55]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import keras
from keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten
from keras import regularizers
from keras import Sequential

In [56]:
#Embedding layer for songs
input_songs = Input(shape=(song_input_dim,), name='songs_input')
embedding_songs = Embedding(input_dim=num_songs+1, output_dim=10, name='embedding_songs')(input_songs)
flatten_songs = Flatten(name='flatten_songs')(embedding_songs)

#Embedding layer for users
input_users = Input(shape=(user_input_dim,), name='users_input')
embedding_users = Embedding(input_dim=num_users+1, output_dim=10, name='embedding_users')(input_users)
flatten_users = Flatten(name='flatten_users')(embedding_users)


#Concatenation and Dense layers
concat_layer = Concatenate(name='concatenate_layer')([flatten_songs, flatten_users])
hidden_layer_1 = Dense(16, activation='relu', name='hidden_layer_1')(concat_layer)
hidden_layer_2 = Dense(8, activation='relu', name='hidden_layer_2')(hidden_layer_1)
output_layer = Dense(1, name='output_layer')(hidden_layer_2)

#Creating model
model1 = Model(inputs=[input_songs, input_users], outputs=output_layer)

#Model compilation
optimizer = Adam(lr=0.001, epsilon=1e-6, amsgrad=True)
model1.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 songs_input (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 users_input (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 embedding_songs (Embedding)    (None, 1, 10)        2671700     ['songs_input[0][0]']            
                                                                                                  
 embedding_users (Embedding)    (None, 1, 10)        834410      ['users_input[0][0]']            
                                                                                            

In [57]:
#Training model

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2)

history = model1.fit(
  [train_song_ids, train_user_ids], train_output, #
  batch_size=256, 
  epochs=50, 
  verbose=1,
  callbacks = [early_stopping],
  validation_split=0.2
)

Epoch 1/50
4587/4587 [==============================] - 76s 16ms/step - loss: 6.1422e-04 - root_mean_squared_error: 0.0248 - val_loss: 5.0931e-04 - val_root_mean_squared_error: 0.0226
Epoch 2/50
4587/4587 [==============================] - 68s 15ms/step - loss: 4.5763e-04 - root_mean_squared_error: 0.0214 - val_loss: 4.9915e-04 - val_root_mean_squared_error: 0.0223
Epoch 3/50
4587/4587 [==============================] - 68s 15ms/step - loss: 3.9838e-04 - root_mean_squared_error: 0.0200 - val_loss: 5.1342e-04 - val_root_mean_squared_error: 0.0227
Epoch 4/50
4587/4587 [==============================] - 63s 14ms/step - loss: 3.1897e-04 - root_mean_squared_error: 0.0179 - val_loss: 5.4300e-04 - val_root_mean_squared_error: 0.0233


In [58]:
from sklearn.metrics import mean_squared_error,r2_score
# Predict on test set
y_pred = model1.predict([test_song_ids,test_user_ids])
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
mae = np.sqrt(mean_absolute_error(test_output, y_pred))

# Evaluate model
score = model1.evaluate([test_song_ids,test_user_ids], test_output, verbose=0)
rmse = np.sqrt(mean_squared_error(test_output, y_pred))
r2 = r2_score(test_output, y_pred)

print('Test mae:',mae)
print('Test rmse:',rmse)
print('Test MSE:', score[0])
print('Test MAE:', score[1])


11466/11466 [==============================] - 16s 1ms/step
Test mae: 0.10702499559895497
Test MSE: 0.0005458444938994944
Test MAE: 0.02336331456899643


In [67]:
print('Test mae:',rmse)


Test mae: 0.0233632893424123


### METADATA

In [59]:
# Convert the table to a Pandas DataFrame
metadata_table_df = pd.read_csv('../datasets/track_metadata.csv')
metadata_table_df.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,NaN,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0


In [60]:
song_ids = taste_profile["song_id"].unique().astype(str)
metadata_df = pd.merge(pd.DataFrame(song_ids,columns=['song_id']),metadata_table_df,on="song_id",how='left')
metadata_df.head()

,song_id,track_id,title,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,SOAKIMP12A8C130995,TRIQAUQ128F42435AD,The Cove,Thicker Than Water,ARC8CQZ1187B98DECA,ff6e677f-91dd-4986-a174-8db0474b1799,Jack Johnson,112.63955,0.832012,0.677482,0,2093263,-1,0
1,SOAPDEY12A81C210A9,TRIRLYL128F42539D1,Nothing from Nothing,To Die For,ARHYUI71187FB48366,8a7cf497-dc5c-4523-932d-3fcbc9a69d38,Billy Preston,153.59955,0.580555,0.482492,1974,3072847,-1,0
2,SOBBMDR12A8C13253B,TRMHBXZ128F4238406,Entre Dos Aguas,Flamenco Para Niños,ARC1SF21187FB51D0F,7bc8dd01-35ea-4b18-a64d-3a364dc76a52,Paco De Lucia,358.24281,0.633119,0.417718,1976,2631392,-1,0
3,SOBFNSP12AF72A0E22,TRYQMNI128F147C1C7,Under Cold Blue Stars,Under Cold Blue Stars,ARMS97V1187B99DD98,023d64c9-93db-4a20-8c5c-2efa1a53481a,Josh Rouse,266.34404,0.675339,0.509058,2002,413957,-1,0
4,SOBFOVM12A58A7D494,TRAHZNE128F9341B86,Riot Radio (Soundtrack Version),Nick & Norah's Infinite Playlist - Original Mo...,ARI4V0E1187B9B6FC2,87ebbe67-3910-4521-a418-4fe53eb912b7,The Dead 60s,141.42649,0.650281,0.451749,0,5674853,-1,0


### PREPROCESSING ON METADATA

In [61]:
metadata_df.replace("",np.nan,inplace=True)

In [62]:
metadata_df.isna().sum()

song_id                  0
track_id                 0
title                    2
release                  0
artist_id                0
artist_mbid           4279
artist_name              0
duration                 0
artist_familiarity       0
artist_hotttnesss        0
year                     0
track_7digitalid         0
shs_perf                 0
shs_work                 0
dtype: int64

In [63]:
values = {'song_hotttnesss': 0,'artist_familiarity': 0, 'title': "Unknown"}
metadata_df.fillna(value= values,inplace=True)
metadata_df.isna().sum()

song_id                  0
track_id                 0
title                    0
release                  0
artist_id                0
artist_mbid           4279
artist_name              0
duration                 0
artist_familiarity       0
artist_hotttnesss        0
year                     0
track_7digitalid         0
shs_perf                 0
shs_work                 0
dtype: int64

In [64]:
metadata_df.columns,metadata_df.dtypes

(Index(['song_id', 'track_id', 'title', 'release', 'artist_id', 'artist_mbid',
        'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
        'year', 'track_7digitalid', 'shs_perf', 'shs_work'],
       dtype='object'),
 song_id                object
 track_id               object
 title                  object
 release                object
 artist_id              object
 artist_mbid            object
 artist_name            object
 duration              float64
 artist_familiarity    float64
 artist_hotttnesss     float64
 year                    int64
 track_7digitalid        int64
 shs_perf                int64
 shs_work                int64
 dtype: object)

In [65]:
metadata_df.columns

Index(['song_id', 'track_id', 'title', 'release', 'artist_id', 'artist_mbid',
       'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
       'year', 'track_7digitalid', 'shs_perf', 'shs_work'],
      dtype='object')

In [66]:
def get_recommendations(user_id,model,taste_profile,metadata_df,data_encoder_user, data_encoder_song):
    song_id_df = pd.DataFrame(taste_profile.song_id.unique(),columns = ['song_id'])
    user_song_df = song_id_df.copy()
    user_song_df['user_id'] = user_id
    encoded_user = data_encoder_user.transform(user_song_df.user_id)
    encoded_song = data_encoder_song.transform(user_song_df.song_id)
    user_song_df['user_id'] = encoded_user
    user_song_df['song_id']= encoded_song
    listened_songs = list(final_data_df[final_data_df.user_id == user_id].song_id.unique())
    songs_to_predict = np.setdiff1d(song_id_df, listened_songs)
    data = pd.merge(user_song_df, final_data_df.drop(columns = 'user_id').drop_duplicates(), on =['song_id'], how = 'left')

    data['Predictions'] = model.predict([data.song_id, data.user_id])
    data['user_id'] = data_encoder_user.inverse_transform(data['user_id'])
    data['song_id'] =data_encoder_song.inverse_transform(data['song_id'])
    top_songs_rec = data[['user_id','song_id','Predictions']].sort_values(by = 'Predictions',ascending = False).song_id[:20].values
    song_info = metadata_df[metadata_df['song_id'].isin(top_songs_rec)][[ 'title', 'artist_name', 'duration']]
    return song_info.reset_index(drop=True)

In [39]:
user_id = 'b80344d063b5ccb3212f76538f3d9e43d87dca9e'
get_recommendations(user_id,model1,taste_profile,metadata_df,data_encoder_user, data_encoder_song)

12139/12139 [==============================] - 15s 1ms/step


,title,artist_name,duration
0,Decades Of Despair,Carnal Forge,182.33424
1,Brother,Keisha White,251.42812
2,Rise Above (Live) (Album Version),Misfits,164.46649
3,Tennessee (Pirate Radio Mix),ARRESTED DEVELOPMENT,371.59138
